In [1]:
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import jaccard_score, multilabel_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
from skopt import BayesSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
import librosa
import librosa.display
import librosa.feature
import IPython.display as ipd

In [2]:
x = np.load('../Part1.npy')
Y_t = np.load('../train.npy/y_train.npy')
#X_train = (x-np.min(x))/(np.max(x)-np.min(x))
xt = np.load('../Part2.npy')
Y_test = np.load('../test.npy/Y_Test.npy')
#X_test = (xt-np.min(xt))/(np.max(xt)-np.min(xt))
x_add = np.load('../Part3.npy')
y_add = np.load('../additional.npy/Y_additional.npy')
#x_add = (x_add-np.min(x_add))/(np.max(x_add)-np.min(x_add))
Y_train = np.concatenate((Y_t, Y_test, y_add))
X_train = np.concatenate((x, xt, x_add))
#X_train = (X_train-np.min(X_train))/(np.max(X_train)-np.min(X_train))
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

In [3]:
X_train.shape

(2874, 17)

In [4]:
#IRMAS-TrainingData/cel/[cel][pop_roc]0143__3.wav
instruments = ['cel/','cla/','flu/','gac/','gel/','org/','pia/','sax/','tru/','vio/','voi/']
pathAudio = "../IRMAS-TrainingData/"
X_test = []
Y_test = []
cnt = 0
for i in instruments:
    label = np.zeros(11)
    label[cnt] = 1
    cnt +=1
    #print(pathAudio + i)
    files = librosa.util.find_files(pathAudio + i, ext=['wav'], recurse=False) 
    files = np.asarray(files)
    #print(files)
    for y in files:
        #print(test)
        y, sr = librosa.load(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        rolloff = np.mean(rolloff)
        bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        bw = np.mean(bw)
        ctr = librosa.feature.spectral_centroid(y=y, sr=sr)
        ctr = np.mean(ctr)
        zcr = librosa.feature.zero_crossing_rate(y=y)
        zcr = np.mean(zcr)
        mfcc = np.mean(mfcc, axis=1)
        mfcc = mfcc[:13]
        mfcc = np.append(mfcc, [rolloff, bw, ctr, zcr])
        X_test.append(mfcc)
        Y_test.append(label)

In [5]:
np.array(X_test).shape

(6705, 17)

In [6]:
mlp = MLPClassifier(activation = 'identity',hidden_layer_sizes = [100] ,solver = 'lbfgs',alpha = 0.00003, batch_size = 50,
                    learning_rate_init=0.058, max_iter = 114, warm_start=True)
chains = [ClassifierChain(mlp, order="random", random_state=i) for i in range(10)]
for chain in chains:
    chain.fit(X_train, Y_train)

In [7]:
Y_pred_chains = np.array([chain.predict(X_test) for chain in chains])
chain_jaccard_scores = [
    jaccard_score(Y_test, Y_pred_chain >= 0.5, average="samples")
    for Y_pred_chain in Y_pred_chains
]

Y_pred_ensemble = Y_pred_chains.mean(axis=0)

In [8]:
chain_jaccard_scores

[0.08014097510741806,
 0.08039771314939101,
 0.07942225063030434,
 0.08,
 0.07913213309186463,
 0.08014417101665423,
 0.0782273356770001,
 0.08000142040410496,
 0.07970668655232414,
 0.08147189375377294]

In [9]:
m = multilabel_confusion_matrix(Y_test, Y_pred_chains[4])

In [10]:
m

array([[[ 277, 6040],
        [  33,  355]],

       [[ 266, 5934],
        [   8,  497]],

       [[  91, 6163],
        [  11,  440]],

       [[6068,    0],
        [ 637,    0]],

       [[5945,    0],
        [ 760,    0]],

       [[   3, 6020],
        [   0,  682]],

       [[   0, 5984],
        [   0,  721]],

       [[6079,    0],
        [ 626,    0]],

       [[5930,  198],
        [ 572,    5]],

       [[6083,   42],
        [ 580,    0]],

       [[5053,  874],
        [ 775,    3]]], dtype=int64)

In [17]:
Y_pred_ensemble

array([[0.9, 1. , 1. , ..., 0. , 0. , 0. ],
       [1. , 1. , 1. , ..., 0. , 0. , 0. ],
       [1. , 1. , 1. , ..., 0. , 0. , 0. ],
       ...,
       [1. , 1. , 1. , ..., 0.3, 0. , 0. ],
       [1. , 1. , 1. , ..., 0.3, 0. , 0. ],
       [1. , 1. , 1. , ..., 0. , 0. , 0. ]])

In [20]:
Y_pred = np.where(Y_pred_ensemble>0.5,1,0)

In [21]:
Y_pred

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [22]:
m2 = multilabel_confusion_matrix(Y_test, Y_pred)

In [23]:
m2

array([[[ 622, 5695],
        [  70,  318]],

       [[ 323, 5877],
        [  10,  495]],

       [[  88, 6166],
        [  11,  440]],

       [[6068,    0],
        [ 637,    0]],

       [[5868,   77],
        [ 745,   15]],

       [[   0, 6023],
        [   0,  682]],

       [[   0, 5984],
        [   0,  721]],

       [[6079,    0],
        [ 626,    0]],

       [[6128,    0],
        [ 577,    0]],

       [[6125,    0],
        [ 580,    0]],

       [[5196,  731],
        [ 775,    3]]], dtype=int64)

In [24]:
precision = [x[1,1]/(x[1,1]+x[0,1]) for x in m]

C:\Users\jonat\AppData\Local\Temp\ipykernel_11520\2312814414.py:1: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = [x[1,1]/(x[1,1]+x[0,1]) for x in m]


In [25]:
precision2 = [x[1,1]/(x[1,1]+x[0,1]) for x in m2]

C:\Users\jonat\AppData\Local\Temp\ipykernel_11520\986516168.py:1: RuntimeWarning: invalid value encountered in longlong_scalars
  precision2 = [x[1,1]/(x[1,1]+x[0,1]) for x in m2]


In [26]:
precision

[0.05551211884284597,
 0.07728191572072772,
 0.0666363774042102,
 nan,
 nan,
 0.10176066845717696,
 0.1075316927665921,
 nan,
 0.024630541871921183,
 0.0,
 0.0034207525655644243]

In [27]:
precision2

[0.052885414934309,
 0.07768361581920905,
 0.06660611565243718,
 nan,
 0.16304347826086957,
 0.10171513795674869,
 0.1075316927665921,
 nan,
 nan,
 nan,
 0.004087193460490463]

In [29]:
np.save(open('X_single.npy', 'wb'), X_test)

In [30]:
np.save(open('Y_single.npy', 'wb'), Y_test)